<div align="right"><i>Peter Norvig<br>2012<br>updated 2019, 2020</i></div>

# Weighing Twelve Balls on a Scale: 🎱🎱🎱🎱 ⚖ 🎱🎱🎱🎱 

> *You are given twelve identical-looking balls and a two-sided scale. One of the balls is of a different weight, although you don't know whether it is lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it is lighter or heavier?*

This is [a classic](https://en.wikipedia.org/wiki/Balance_puzzle) brain-teaser puzzle (first appearing in 1945), meant to be solved with paper and pencil.  

However I want to solve not just this specific puzzle, but related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the different ball might be heavier, lighter, or either, or neither. So I'll be better served by a computer program, not a pencil. 

I originally wrote this program in Lisp around 1980, ported it to Python in 2012, and am republishing it here in revised form after I saw the puzzle was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019. It also appeared in a [Numberplay](https://wordplay.blogs.nytimes.com/2014/07/21/12coin/) column in 2014 (with coins instead of balls), in the Museum of Math's [Mind-Benders for the Quarantined!](https://momath.org/civicrm/?page=CiviCRM&q=civicrm/event/info&reset=1&id=1620) series in 2020, and in many other venues.


# Defining the Vocabulary
    
Here are the concepts I'm dealing with:

- **ball**: A ball is represented by a positive integer. A list of balls by a list, e.g. `[1, 2, 3]`.
- **oddball**: One of the balls is different in weight; I'll use the term **oddball** to indicate which ball is different, and how. I'll use `+N` to mean that ball `N` is heavier, `-N` to mean ball `N` is lighter, and `0` to mean that all the balls are the same. (So `0` cannot be a ball number.) 
- **puzzle**: I'll express the puzzle stated above with `Puzzle(N=12, weighings=3)`. A puzzle is defined by the number of balls, the number of weighings allowed, and the possible oddballs (I'll say how to describe them later).
- **weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result will be that the left side is greater than, equal to, or less than the right in weight. I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **weight**: I'll arbitrarily say that a normal ball weighs 100, a lighter ball 99, and a heavier ball 101.
- **solution**: Given a puzzle, a solution is a **strategy tree** that can correctly discover the oddball, whatever it is, in the allowable number of weighings. `valid(tree, puzzle)` is true when the tree is a valid solution for the puzzle. `None` indicates the failure to find a solution.
- **strategy tree**: A tree whose leaf nodes are oddballs and whose interior nodes have 5 components:  `Node(L, R, gt, eq, lt)` is a node where `L` and `R` are the lists of balls to be weighed, and `gt`, `eq`, and `lt` are branches for subtrees corresponding to the three possible weighing results. 
- **following a path**: I'll use `follow(tree, oddball)` to say *follow the path through the tree, doing each weighing under the assumption of the given oddball, and return the leaf node reached by the path&mdash;the oddball that the tree predicts.* Note that the function `follow` gets to see what the oddball is, but the `tree` does not; the tree has to figure it out by doing weighings.



# Implementation of Data Types

Most of the types are straightforward. In a call to `Puzzle(...)` class, you can specify the allowable oddballs in two ways:
- Use `oddballs={...}` to specify an explicit set of oddballs: any set of positive, negative, or zero integers.
- Use `odd={...}`, specifying a subset of `{-1, 0, +1}`, where `-1` means "any one ball might be lighter," `+1` means "any one ball might be heavier," and `0` means "it might be that no ball is odd; they might all weigh the same." The default is `odd={-1, +1}`, meaning that the oddball can be any ball, and can be lighter or heavier.



In [1]:
from collections import namedtuple
import random

Ball = int            # Balls are positive integers
Leaf = Oddball = int  # Oddballs are positive, negative, or zero integers
Node = namedtuple('Node', 'L, R, gt, eq, lt') # A Node is a tuple of 5 components
Tree = (Node, Leaf)   # A tree can be an interior node or a leaf

class Puzzle:
    "Represent a specific ball-weighing puzzle."
    def __init__(self, N=12, weighings=3, odd={-1, +1}, oddballs=None):
        self.balls     = list(range(1, N + 1))
        self.weighings = weighings
        self.oddballs  = oddballs or {b * o for b in self.balls for o in odd}

# Implementation of Basic Functions

Here are all the straightforward utility functions; everything except the main function `find_tree` that actually solves a puzzle.

In [2]:
def weigh(L, R, oddball) -> str:
    "Weigh balls L against balls R, given the oddball; return 'gt', 'eq', or 'lt'."
    Δ = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return 'gt' if Δ > 0 else 'lt' if Δ < 0 else 'eq'

def weight(ball, oddball) -> int: 
    "How heavy is this ball (given that we know what the oddball is)?"
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Tree or None:
    "Return a valid tree (one that solves the puzzle), or None for failure."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    return tree if valid(tree, puzzle) else None
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if is_leaf(tree):
        return tree
    else:
        branch = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, branch), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (tree is not None and
            depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth (number of weighings) in tree."
    return (0 if is_leaf(tree) or tree is None
            else 1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

def first(iterable): return next(iter(iterable))

def is_leaf(tree): return isinstance(tree, Leaf)

Let's try out some of these functions:

In [3]:
# A puzzle with 8 balls and 3 weighings allowed
p8 = Puzzle(8, 3) 
p8.weighings

3

In [4]:
p8.balls 

[1, 2, 3, 4, 5, 6, 7, 8]

In [5]:
p8.oddballs

{-8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8}

In [6]:
# If we weigh balls [1, 2] against [3, 4] and 4 is lighter, the result should be 'gt'
weigh([1, 2], [3, 4], -4)

'gt'

In [7]:
weight(1, oddball=-4)

100

In [8]:
weight(4, oddball=-4)

99

# Finding a Valid Solution Tree

Now we've got all the pieces we need to find a valid tree to solve a puzzle. The key concept is that a **weighing** gives us information by making a **partition** of the possible **oddballs** into branches for each of the three possible weighing results: `gt`, `eq`, or `lt`. If each of the partitions is small enough, subsequent weighings can handle each of them. Here's what I mean by a partition:

In [9]:
def partition(L, R, oddballs) -> dict:
    "A dict that partitions the possible outcomes from weighing L versus R."
    part = dict(gt=set(), eq=set(), lt=set())
    for odd in oddballs:
        part[weigh(L, R, odd)].add(odd)
    return part

For example, with 12 balls, if we weigh balls [1, 2, 3] on the left versus [10, 11, 12] on the right, then there are 6 ways  the left side can be greater than the right: either 10, 11 or 12 is lighter or  1, 2, or 3 is heavier. Similarly there are 6 ways of getting a `lt` weighing result. The remaining 12 possibilities&mdash;balls 4 through 9 being either heavier or lighter&mdash;show up in the `eq` branch of the partition:

In [10]:
p12 = Puzzle(12, 3)

partition([1, 2, 3], [10, 11, 12], p12.oddballs)

{'gt': {-12, -11, -10, 1, 2, 3},
 'eq': {-9, -8, -7, -6, -5, -4, 4, 5, 6, 7, 8, 9},
 'lt': {-3, -2, -1, 10, 11, 12}}

If this was the first weighing in our strategy tree, could we go on to solve the puzzle in 3 weighings? 

**No!** The key insight is that a tree with <i>w</i> weighings can distinguish at most  3<sup><i>w</i></sup> oddballs, because each weighing has three possible results. After using one weighing we can only solve the puzzle if every branch of the partition has no more than 3<sup>2</sup> = 9 oddballs, but here we have 12 oddballs in the `eq` branch. We call this a **bad partition**. A good partition is one where no branch has more than 3<sup><i>w</i></sup> oddballs. Being a good partition does not guarantee that the puzzle is solvable from there; but being a bad partition guarantees that it is not solvable.
 

In [11]:
def good_partition(part, w) -> bool:
    "Is this a partition which might lead to a solution in w more weighings?"
    return all(len(oddballs) <= 3 ** w for oddballs in part.values())

The following is a good partition because each of the branches has 8 possibilities, and 8 is less than 3<sup>2</sup>:

In [12]:
p = partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)
p

{'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
 'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
 'lt': {-4, -3, -2, -1, 9, 10, 11, 12}}

In [13]:
good_partition(p, 2)

True

So now we have a viable approach to implementing `find_tree`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we use the oddballs from the partition, and reduce the number of remaining weighings by 1.
   - We handle trivial cases when there are zero or one oddballs remaining, or no weighings remaining.
   - Otherwise, we ask `good_partitions` to generate random partitions and yield the good ones.
   - Given a good partition, try to find valid trees for each of the three branches. If we can find that, return the tree as a solution.
   - If we can't find good trees for any partition, return `None` to indicate failure.
   
The rest of the work is done by `good_partitions`:

   - Select two lists of balls to be weighed, `L` and `R`, both of length `B`.
   - See what partition `L` and `R` gives us.
   - If it is good, yield the `(L, R, partition)` tuple.
   - Repeat for all possible values of `B`.
   - Randomly shuffle the balls and repeat the whole process, trying to generate more good partitions. I chose to repeat 100 times, to have a good chance of finding a solution for solvable puzzles, and of terminating in just a few seconds for unsolvable puzzles. 
   - However, I note that on the first weighing, there's no sense making multiple tries with randomly shuffled  balls&mdash;the only choice that matters is how many balls, `B`, to put on each side, because the balls are undifferentiated at this point. So I try each value of `B` only once on the first weighing.
   - If I wanted to solve puzzles with thousands of balls, not just dozens, I would come back and improve `good_partitions`, passing information from the first weighing in to subsequent weighings.

In [14]:
def find_tree(puzzle, oddballs, weighings) -> Tree or None:
    "Find a tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return first(oddballs) # One possibility left; we're done; leaf node
    elif len(oddballs) == 0:
        return 0               # No oddball (a possibility for some puzzles)
    elif weighings == 0:
        return None            # No solution; failure
    else:
        for L, R, part in good_partitions(puzzle, oddballs, weighings - 1):
            subtrees = {r: find_tree(puzzle, part[r], weighings - 1) for r in part}
            if None not in subtrees.values(): 
                return Node(L, R, **subtrees)
    
def good_partitions(puzzle, oddballs, weighings, tries=100):
    "Yield random (L, R, partition) tuples of good partitions."
    balls = sorted(puzzle.balls)
    if weighings == puzzle.weighings: tries = 1 # First weighing
    for _ in range(tries): 
        for B in range(1, 1 + len(balls) // 2):
            L, R = balls[:B], balls[-B:]
            part = partition(L, R, oddballs)
            if good_partition(part, weighings):
                yield L, R, part
        random.shuffle(balls)

Here we see that  `good_partitions` does its job:

In [15]:
next(good_partitions(p12, p12.oddballs, 2))

([1, 2, 3, 4],
 [9, 10, 11, 12],
 {'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
  'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
  'lt': {-4, -3, -2, -1, 9, 10, 11, 12}})

# Solving Some Puzzles

Now we're ready to solve puzzles! We'll start with the original 12-ball puzzle, `p12`:

In [16]:
solve(p12)

Node(L=[1, 2, 3, 4], R=[9, 10, 11, 12], gt=Node(L=[11, 3, 10, 2], R=[6, 12, 8, 5], gt=Node(L=[9, 4, 10, 8], R=[12, 5, 6, 3], gt=-12, eq=2, lt=3), eq=Node(L=[8, 6], R=[1, 9], gt=-9, eq=4, lt=1), lt=Node(L=[1, 2], R=[11, 12], gt=-11, eq=-10, lt=0)), eq=Node(L=[12, 1, 10, 9, 4], R=[5, 11, 6, 8, 2], gt=Node(L=[1, 2, 3, 4, 5], R=[8, 9, 10, 11, 12], gt=-8, eq=-6, lt=-5), eq=Node(L=[1, 2, 3, 4, 5, 6], R=[7, 8, 9, 10, 11, 12], gt=-7, eq=0, lt=7), lt=Node(L=[1, 2, 3, 4, 5], R=[8, 9, 10, 11, 12], gt=5, eq=6, lt=8)), lt=Node(L=[11, 2, 7, 8], R=[1, 6, 3, 10], gt=Node(L=[10, 6], R=[1, 11], gt=-1, eq=-3, lt=11), eq=Node(L=[6, 7, 12, 4], R=[3, 5, 8, 10], gt=12, eq=9, lt=-4), lt=Node(L=[1, 2], R=[11, 12], gt=0, eq=10, lt=-2)))

OK, that's hard to read&mdash;my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the oddball can only be heavier:

In [17]:
tree = solve(Puzzle(3, 1, odd={+1}))
tree

Node(L=[1], R=[3], gt=1, eq=2, lt=3)

This trivial tree says you weigh the first ball against the third (leaving the second unweighed), and the three possible weighing results tell you which of the three balls  is heavier. 

# Prettier Output

Let's make the output easier to read. I'll use `[1·2·3·4]⚖[9·10·11·12]` to represent a weighing and I'll use `>:` to represent a branch. 

In [18]:
def do(puzzle):
    "Print the solution to the puzzle as indented text."
    tree = solve(puzzle)
    print(indented(tree))
    
def indented(tree, i=0, prefix='') -> str:
    "Pretty, indented string representing a strategy tree."
    if tree == 0 or tree == None:
        return f'{prefix}{tree}'
    elif is_leaf(tree):
        return f'{prefix}{tree:+d}'
    else:
        subtrees = (indented(tree.gt, i+1, '>:'), 
                    indented(tree.eq, i+1, '=:'), 
                    indented(tree.lt, i+1, '<:'))
        indent   = ('' if i == 0 else ('\n' + ' ' * 3 * i))
        return f'{indent}{prefix}{side(tree.L)}⚖{side(tree.R)} ➔ {" ".join(subtrees)}'
    
def side(balls) -> str: return str(sorted(balls)).replace(', ', '·')

In [19]:
# 3 balls, 1 weighing, only heavier balls possible
do(Puzzle(3, 1, {+1}))

[1]⚖[3] ➔ >:+1 =:+2 <:+3


In [20]:
# 3 balls, 2 weighings, lighter or heavier balls possible
do(Puzzle(3, 2))

[1]⚖[3] ➔ 
   >:[1]⚖[2] ➔ >:+1 =:-3 <:0 
   =:[3]⚖[2] ➔ >:-2 =:0 <:+2 
   <:[3]⚖[2] ➔ >:+3 =:-1 <:0


In [21]:
# 3 balls, 1 weighings, either ball 1 is heavy or 2 or 3 is light
do(Puzzle(3, 1, oddballs={+1, -2, -3}))

[3]⚖[2] ➔ >:-2 =:+1 <:-3


In [22]:
# The original puzzle with 12 balls and 3 weighings
do(p12)

[1·2·3·4]⚖[9·10·11·12] ➔ 
   >:[2·4·10]⚖[1·5·12] ➔ 
      >:[1·2·5·8·11]⚖[3·4·6·7·9] ➔ >:+2 =:-12 <:+4 
      =:[1·4·7·11·12]⚖[2·5·6·9·10] ➔ >:-9 =:+3 <:-11 
      <:[1]⚖[12] ➔ >:+1 =:-10 <:0 
   =:[2·3·6·10]⚖[5·8·11·12] ➔ 
      >:[1·2·3·4·5]⚖[8·9·10·11·12] ➔ >:-8 =:+6 <:-5 
      =:[1·2·3·4·5·6]⚖[7·8·9·10·11·12] ➔ >:-7 =:0 <:+7 
      <:[1·2·3·4·5]⚖[8·9·10·11·12] ➔ >:+5 =:-6 <:+8 
   <:[3·4·7·11]⚖[2·6·8·10] ➔ 
      >:[8·12]⚖[2·9] ➔ >:-2 =:+11 <:0 
      =:[3·4·6·9·11]⚖[2·5·7·10·12] ➔ >:+9 =:-1 <:+12 
      <:[4·5·6·12]⚖[1·3·8·9] ➔ >:-3 =:+10 <:-4


In [23]:
# A different random solution to the same puzzle
do(p12)

[1·2·3·4]⚖[9·10·11·12] ➔ 
   >:[2·11·12]⚖[1·8·9] ➔ 
      >:[1·2]⚖[11·12] ➔ >:+2 =:-9 <:0 
      =:[6·7]⚖[3·10] ➔ >:-10 =:+4 <:+3 
      <:[1·3·8·10·11]⚖[2·4·5·7·9] ➔ >:+1 =:-12 <:-11 
   =:[5·9·10·11·12]⚖[1·3·4·6·7] ➔ 
      >:[1·3·4·9·12]⚖[5·7·8·10·11] ➔ >:-7 =:-6 <:+5 
      =:[1·2·3·4·5]⚖[8·9·10·11·12] ➔ >:-8 =:0 <:+8 
      <:[3·4·7]⚖[6·9·11] ➔ >:+7 =:-5 <:+6 
   <:[3·6·9·10]⚖[2·5·8·12] ➔ 
      >:[2·5·8·9]⚖[1·4·6·11] ➔ >:+9 =:+10 <:-2 
      =:[3·6·7·12]⚖[2·4·10·11] ➔ >:-4 =:-1 <:+11 
      <:[1]⚖[12] ➔ >:0 =:-3 <:+12


I note that the traditional answer to the puzzle weighs 4 balls on each side of the first weighing, three balls on the second weighing, and 2 balls on the third weighing. But my program is not so orderly. It always weighs 4 balls on the first weighing (because that is the only good partition), but from there it can vary; it need not minimize the number of balls on each side of the scale, nor make one branch of the tree be symmetric with another branch.

# Other Puzzles: 3 Weighings



We can do **12 balls in 3 weighings** even when we add in the possibility that all the balls weigh the same:

In [24]:
do(Puzzle(12, 3, odd={-1, 0, +1}))

[1·2·3·4]⚖[9·10·11·12] ➔ 
   >:[4·7·11·12]⚖[1·5·6·9] ➔ 
      >:[6·9]⚖[7·8] ➔ >:0 =:+4 <:-9 
      =:[2·6·9·11]⚖[1·3·4·7] ➔ >:+2 =:-10 <:+3 
      <:[1·4·12]⚖[6·7·10] ➔ >:+1 =:-11 <:-12 
   =:[1·4·9]⚖[5·7·8] ➔ 
      >:[1·2·3·4·5]⚖[8·9·10·11·12] ➔ >:-8 =:-7 <:-5 
      =:[1·2·3·4·5·6]⚖[7·8·9·10·11·12] ➔ >:+6 =:0 <:-6 
      <:[1·2·3·4·5]⚖[8·9·10·11·12] ➔ >:+5 =:+7 <:+8 
   <:[1·4·12]⚖[2·7·10] ➔ 
      >:[1]⚖[12] ➔ >:0 =:-2 <:+12 
      =:[2·6·10·11·12]⚖[1·5·7·8·9] ➔ >:+11 =:-3 <:+9 
      <:[7·8·11]⚖[4·5·10] ➔ >:-4 =:-1 <:+10


Can we solve the **13-balls in 3 weighings** puzzle, which has 26 possibilities? After all, 26 is less than 3<sup>3</sup>.

In [25]:
p13 = Puzzle(13, 3)
do(p13)

None


**No**, and the reason is that there is no first weighing that partitions the 26 possibilities into 9/9/8; the best we can do is partition into 8/10/8 or 10/6/10, and 10 possibilities can't be distinguished in the remaining two weighings:

In [26]:
partition([1, 2, 3, 4], [10, 11, 12, 13], p13.oddballs)

{'gt': {-13, -12, -11, -10, 1, 2, 3, 4},
 'eq': {-9, -8, -7, -6, -5, 5, 6, 7, 8, 9},
 'lt': {-4, -3, -2, -1, 10, 11, 12, 13}}

In [27]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], p13.oddballs)

{'gt': {-13, -12, -11, -10, -9, 1, 2, 3, 4, 5},
 'eq': {-8, -7, -6, 6, 7, 8},
 'lt': {-5, -4, -3, -2, -1, 9, 10, 11, 12, 13}}

We can do **27 balls in 3 weighings** if we know that the oddball can only be lighter, not heavier:

In [28]:
do(Puzzle(27, 3, {-1}))

[1·2·3·4·5·6·7·8·9]⚖[19·20·21·22·23·24·25·26·27] ➔ 
   >:[1·5·6·7·14·16·18·19·23·25]⚖[2·8·10·11·12·15·17·21·24·26] ➔ 
      >:[2·10·11·12·18·24]⚖[5·7·13·16·26·27] ➔ >:-26 =:-21 <:-24 
      =:[1·2·4·12·20·24·25]⚖[6·7·8·13·14·17·22] ➔ >:-22 =:-27 <:-20 
      <:[19]⚖[23] ➔ >:-23 =:-25 <:-19 
   =:[1·2·3·4·5·6·7·8·9·10·11·12]⚖[16·17·18·19·20·21·22·23·24·25·26·27] ➔ 
      >:[2·5·12·16·20·22·24]⚖[4·9·17·19·21·23·25] ➔ >:-17 =:-18 <:-16 
      =:[1·2·3·4·5·6·7·8·9·10·11·12·13]⚖[15·16·17·18·19·20·21·22·23·24·25·26·27] ➔ >:-15 =:-14 <:-13 
      <:[10·17·20]⚖[11·14·21] ➔ >:-11 =:-12 <:-10 
   <:[2·4·9·12·16·17·20·23·25·26]⚖[1·5·8·10·11·13·14·19·21·22] ➔ 
      >:[1·2·3·7·10·12·13·14·15·21·25]⚖[6·8·9·11·16·17·18·19·22·23·24] ➔ >:-8 =:-5 <:-1 
      =:[3·5·11·15·17·24·26]⚖[7·8·14·18·19·22·23] ➔ >:-7 =:-6 <:-3 
      <:[4·5·8·10·15·16·18·20·27]⚖[1·2·11·12·17·21·23·24·25] ➔ >:-2 =:-9 <:-4


And we can do **26 balls** under the condition that either one ball is lighter or all the balls weigh the same:

In [29]:
do(Puzzle(26, 3, {-1, 0}))

[1·2·3·4·5·6·7·8·9]⚖[18·19·20·21·22·23·24·25·26] ➔ 
   >:[1·5·10·12·16·18·21·22]⚖[3·9·11·14·15·19·23·24] ➔ 
      >:[2·9·14·17·20·23]⚖[3·4·13·16·19·21] ➔ >:-19 =:-24 <:-23 
      =:[3·7·15·17·20·22]⚖[11·14·16·21·24·25] ➔ >:-25 =:-26 <:-20 
      <:[3·5·7·21]⚖[2·9·14·22] ➔ >:-22 =:-18 <:-21 
   =:[1·2·3·4·5·6·7·8·9·10·11·12]⚖[15·16·17·18·19·20·21·22·23·24·25·26] ➔ 
      >:[8·16·18·21]⚖[1·2·17·20] ➔ >:-17 =:-15 <:-16 
      =:[1·2·3·4·5·6·7·8·9·10·11·12·13]⚖[14·15·16·17·18·19·20·21·22·23·24·25·26] ➔ >:-14 =:0 <:-13 
      <:[1·2·4·10·13·15·22·23]⚖[6·7·8·12·16·17·20·21] ➔ >:-12 =:-11 <:-10 
   <:[6·7·9·13·15·20·22·23·26]⚖[1·2·3·17·18·19·21·24·25] ➔ 
      >:[1·7·9·11·16·18·19·21·24·26]⚖[2·4·5·8·10·12·14·22·23·25] ➔ >:-2 =:-3 <:-1 
      =:[2·3·4·6·9·11·14·15·16·17·22]⚖[1·7·8·10·13·18·20·21·23·24·25] ➔ >:-8 =:-5 <:-4 
      <:[1·3·4·9]⚖[6·14·20·25] ➔ >:-6 =:-7 <:-9


Here's a puzzle with **26 balls** where either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or no ball is odd.

In [30]:
N = 26
p26 = Puzzle(N, 3, oddballs={(+b if b % 2 else -b) for b in range(N + 1)})
print(p26.oddballs)

{0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, -26, -24, -22, -20, -18, -16, -14, -12, -10, -8, -6, -4, -2}


In [31]:
do(p26)

[1·8·15·16·17·23·24·25·26]⚖[4·5·7·9·10·11·18·19·20] ➔ 
   >:[1·2·3·4·5·6·7·8·9]⚖[18·19·20·21·22·23·24·25·26] ➔ 
      >:[1·13·18]⚖[5·23·24] ➔ >:+1 =:-20 <:-18 
      =:[3·6·8·9·11·19·20·23·25]⚖[4·10·12·14·17·18·21·22·26] ➔ >:-10 =:+15 <:+17 
      <:[2·8·10·11·16·17·23·24]⚖[1·5·9·12·14·18·25·26] ➔ >:+23 =:-4 <:+25 
   =:[3·4·6·14·18·19·21·23·24·25]⚖[1·5·8·9·12·13·16·17·20·26] ➔ 
      >:[1·2·3·4·5·6]⚖[21·22·23·24·25·26] ➔ >:+3 =:-12 <:+21 
      =:[1·2·3·4·5]⚖[22·23·24·25·26] ➔ >:-22 =:0 <:-2 
      <:[3·6·8·9·15·23·26]⚖[1·2·10·11·14·17·20] ➔ >:-14 =:+13 <:-6 
   <:[3·5·8·20·24]⚖[9·10·12·16·26] ➔ 
      >:[4·14·16·18]⚖[7·17·20·26] ➔ >:-26 =:+5 <:-16 
      =:[1·2·3·4·5·6·7·8]⚖[19·20·21·22·23·24·25·26] ➔ >:+7 =:+11 <:+19 
      <:[1·2·3·4·5·6·7·8]⚖[19·20·21·22·23·24·25·26] ➔ >:-24 =:+9 <:-8


Another variation: **27 balls, 3 weighings**, the oddball can only be heavier, but one ball (number 27) is poisonous and can't be touched or placed on the balance scale. It might, however be the heavier ball, and you still need to report it as such if it is. 

We describe this situation by defining a puzzle with 26 (weighable) balls, but with the oddballs covering `{+1, +2, ..., +27}`. Note that when all three weighings are equal, the strategy tree reports `=:+27`.

In [32]:
do(Puzzle(26, 3, oddballs=range(1, 28)))

[1·2·3·4·5·6·7·8·9]⚖[18·19·20·21·22·23·24·25·26] ➔ 
   >:[2·6·7·18·21]⚖[3·4·5·13·14] ➔ 
      >:[7·8·16·18·20·25·26]⚖[2·3·12·13·15·17·19] ➔ >:+7 =:+6 <:+2 
      =:[2·3·6·7·9·17·26]⚖[4·5·8·13·18·24·25] ➔ >:+9 =:+1 <:+8 
      <:[3·8·11·12·16·17·19·21·22]⚖[1·5·6·7·9·10·13·23·26] ➔ >:+3 =:+4 <:+5 
   =:[1·2·3·4·5·6·7·8·9·10·11·12]⚖[15·16·17·18·19·20·21·22·23·24·25·26] ➔ 
      >:[4·11]⚖[12·21] ➔ >:+11 =:+10 <:+12 
      =:[1·2·3·4·5·6·7·8·9·10·11·12·13]⚖[14·15·16·17·18·19·20·21·22·23·24·25·26] ➔ >:+13 =:+27 <:+14 
      <:[3·4·13·17·18·23·26]⚖[2·5·7·8·10·15·21] ➔ >:+17 =:+16 <:+15 
   <:[1·4·7·9·12·15·18·21·23]⚖[5·6·8·14·16·17·20·22·24] ➔ 
      >:[5·8·10·12·19·20·21]⚖[1·6·9·15·17·23·25] ➔ >:+21 =:+18 <:+23 
      =:[11·19·21]⚖[7·13·25] ➔ >:+19 =:+26 <:+25 
      <:[1·2·3·6·7·8·9·10·12·14·19·24]⚖[4·5·11·13·15·17·18·21·22·23·25·26] ➔ >:+24 =:+20 <:+22


# 4 Weighings

We can tackle larger puzzles. With 4 weighings, we can theoretically handle up to 3<sup>4</sup> = 81 possibilities. Can we solve for **40 balls**?

In [33]:
p40 = Puzzle(40, 4)

do(p40)

None


Unfortunately, **no**, we can't find a good partition, which would leave no more than 27 oddballs in each branch. I'll introduce the function `partition_sizes` so you don't have to count:

In [34]:
def partition_sizes(puzzle, B):
    "How big is each branch of a partition with B balls on both sides?"
    part = partition(puzzle.balls[:B], puzzle.balls[-B:], puzzle.oddballs)
    return [len(oddballs) for oddballs in part.values()]

In [35]:
partition_sizes(p40, 13)

[26, 28, 26]

In [36]:
partition_sizes(p40, 14)

[28, 24, 28]

How about **39 balls, 4 weighings**, heavier or lighter, with the possibility that no ball is odd. That's 39 &times; 2 + 1 = 79 possibilities, which is less than 81.

In [37]:
p39 = Puzzle(39, 4, {-1, 0, +1})

In [38]:
partition_sizes(p39, 13)

[26, 27, 26]

In [39]:
do(p39)

[1·2·3·4·5·6·7·8·9·10·11·12·13]⚖[27·28·29·30·31·32·33·34·35·36·37·38·39] ➔ 
   >:[2·4·9·14·15·16·17·19·26·35·37·38·39]⚖[1·5·6·7·11·23·24·25·27·28·30·31·34] ➔ 
      >:[1·3·4·7·11·21·22·23·31·35·36·38]⚖[2·6·9·12·16·18·20·26·32·33·34·37] ➔ 
         >:[1·2·3·4]⚖[36·37·38·39] ➔ >:+4 =:-34 <:0 
         =:[13·16·19·22·30·33]⚖[1·2·17·20·27·38] ➔ >:-27 =:-28 <:-30 
         <:[3·6·7·10·18·21·30]⚖[9·13·14·19·31·32·38] ➔ >:-31 =:+2 <:+9 
      =:[8·12·19·25·27·28·32·33]⚖[1·7·10·18·20·29·34·37] ➔ 
         >:[2·5·8·9·16·18·22·25·33·36·38]⚖[1·3·6·10·12·14·24·28·31·32·39] ➔ >:+8 =:-29 <:+12 
         =:[7·9·13·14·15·23·25·31·36·38]⚖[5·6·10·12·18·21·26·28·29·32] ➔ >:+13 =:+3 <:-36 
         <:[8·17·20·21·22·35·36]⚖[4·10·19·30·33·34·37] ➔ >:-33 =:-32 <:+10 
      <:[1·3·11·17·18·24·28·30·34·35·37]⚖[2·7·8·9·10·16·23·27·31·33·39] ➔ 
         >:[1·3·7·14·18·21·22·23·25·26·27·33·35·39]⚖[2·4·6·8·9·10·12·13·16·24·28·29·31·36] ➔ >:+1 =:+11 <:-39 
         =:[2·6·8·10·13·14·24·27·39]⚖[5·9·12·16·17·21·31·33

How about **80 balls, 4 weighings** under the condition that no ball can be heavier (thus, 81 possibilities, the maximum):

In [40]:
do(Puzzle(80, 4, {-1, 0}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27]⚖[54·55·56·57·58·59·60·61·62·63·64·65·66·67·68·69·70·71·72·73·74·75·76·77·78·79·80] ➔ 
   >:[1·2·3·5·6·12·15·16·18·20·21·23·26·32·33·38·42·43·44·45·51·53·56·59·60·64·65·66·72·74·79]⚖[4·7·9·11·13·17·22·24·25·30·31·34·35·39·40·41·46·47·48·49·50·52·54·55·58·61·63·67·76·77·80] ➔ 
      >:[1·3·5·6·8·14·20·22·24·25·27·32·33·34·39·43·44·45·46·54·56·57·58·62·63·66·72·74·78·79]⚖[2·9·11·12·15·16·18·19·21·23·26·28·29·30·31·38·40·42·48·50·64·65·67·68·69·70·71·75·77·80] ➔ 
         >:[2·5·8·11·17·22·27·37·39·40·42·49·52·58·65·74·76·80]⚖[3·4·12·14·15·18·26·35·43·45·46·48·60·62·67·69·70·71] ➔ >:-67 =:-77 <:-80 
         =:[4·6·7·8·10·12·13·16·20·21·22·25·27·28·32·38·42·43·47·48·51·53·55·57·58·59·64·65·67·74·77·79]⚖[2·3·5·11·15·18·19·23·24·26·29·30·31·33·34·36·44·45·50·52·56·60·61·62·63·68·69·70·72·75·78·80] ➔ >:-61 =:-76 <:-55 
         <:[5·6·7·9·12·13·14·17·19·21·26·38·39·41·42·46·54·72·77]⚖[1·8·15·16·24·25·36·48·50·51·52·53·5

Looking good (except that the output is longer than the line length, and so is hard to read).

# 5 Weighings

With 5 weighings, 3<sup>5</sup> = 243, so I might be able to handle up to 121 lighter-or-heavier balls (242 possibilities), if I can make partitions of size 81 or less. Let's try:

In [41]:
p5 = Puzzle(121, 5)
do(p5)

None


In [42]:
partition_sizes(p5, 40)

[80, 82, 80]

In [43]:
partition_sizes(p5, 41)

[82, 78, 82]

Nope, that didn't work. Let's back off to **120 balls, 5 weighings** for 2 &times; 120 + 1 = 241 possibilities:

In [44]:
do(Puzzle(120, 5, {-1, 0, +1}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27·28·29·30·31·32·33·34·35·36·37·38·39·40]⚖[81·82·83·84·85·86·87·88·89·90·91·92·93·94·95·96·97·98·99·100·101·102·103·104·105·106·107·108·109·110·111·112·113·114·115·116·117·118·119·120] ➔ 
   >:[2·3·5·8·10·18·21·26·34·35·36·37·47·51·52·53·59·62·66·69·72·74·76·77·82·85·92·99·102·106·107·110·112·113·115·116·117·119]⚖[6·7·11·15·17·20·23·25·27·29·31·39·40·42·49·50·54·57·60·64·70·79·80·84·86·88·89·90·91·93·96·97·98·100·104·105·109·114] ➔ 
      >:[5·11·17·18·21·22·26·27·28·29·30·33·37·39·41·43·44·47·48·51·54·57·58·64·69·71·74·75·76·84·88·89·91·94·100·106·108·109·118]⚖[1·2·6·8·9·12·14·15·16·20·25·31·32·34·42·45·46·50·53·56·62·63·72·77·78·80·81·82·83·87·90·93·95·96·97·99·101·107·116] ➔ 
         >:[2·5·6·8·13·16·17·19·20·28·29·33·34·35·38·40·41·43·44·45·47·53·54·55·56·63·64·67·68·76·82·83·86·91·94·96·97·98·100·102·103·104·107·109·110·113·115·119]⚖[3·4·7·10·14·18·23·27·31·32·39·46·49·50·57·58·59·60·61·62·65·69·71·72·73·74·75

That works.

Or, I could solve a puzzle with 243 possibilites (the maximum possible): **242 balls**, lighter-only, plus the possibility that no ball is odd:

In [45]:
do(Puzzle(242, 5, {-1, 0}))

[1·2·3·4·5·6·7·8·9·10·11·12·13·14·15·16·17·18·19·20·21·22·23·24·25·26·27·28·29·30·31·32·33·34·35·36·37·38·39·40·41·42·43·44·45·46·47·48·49·50·51·52·53·54·55·56·57·58·59·60·61·62·63·64·65·66·67·68·69·70·71·72·73·74·75·76·77·78·79·80·81]⚖[162·163·164·165·166·167·168·169·170·171·172·173·174·175·176·177·178·179·180·181·182·183·184·185·186·187·188·189·190·191·192·193·194·195·196·197·198·199·200·201·202·203·204·205·206·207·208·209·210·211·212·213·214·215·216·217·218·219·220·221·222·223·224·225·226·227·228·229·230·231·232·233·234·235·236·237·238·239·240·241·242] ➔ 
   >:[1·4·6·13·14·17·18·20·22·23·27·31·37·41·53·54·55·56·58·62·63·64·67·80·81·83·86·87·89·91·94·97·98·101·106·107·109·110·111·113·115·125·131·134·135·136·138·139·141·145·146·147·148·150·152·153·156·160·161·163·167·168·169·170·171·173·185·187·190·192·195·199·203·205·207·213·219·223·225·226·231·232·236·238·240·241]⚖[2·3·5·7·9·12·15·19·21·24·25·28·29·30·33·39·44·48·50·52·66·69·70·73·75·76·77·78·79·82·85·93·95·96·99·100·105·108·112·114

# What's Next?

- What other puzzles can you solve?
- Can you make a table summarizing the space of solved and unsolved puzzles? 
- Can you *prove* the unsolved puzzles are unsolvable? 
- Currently, when `solve` returns `None`, it means "no solution was found, but there's no proof that a solution doesn't exist." Can you modify `solve` to return a result that indicates there is no possible solution? (Maybe you can only do this on some puzzles, not all.)
- What about puzzles where your task is to identify *which* ball is odd, but not *how* it is odd? That is, if you get the possibilities down to `{-3, +3}` then you're done; you know ball 3 is the odd one, and you don't care if it is heavy or light.
- Can you find trees that minimize the *mean* number of weighings, rather than minimizing the *maximum* number of weighings?
- What happens when it is a possibility that *two* balls are odd?  Or more?
- What if you had two or more balance scales that you could use in parallel, and the goal was to minimize the number of weighing time periods, not the total number of weighings?
- What else can you discover?